In [2]:
!git clone https://github.com/hungpham13/federated-learning
%cd federated-learning
!pip install -q -r requirements.txt

Cloning into 'federated-learning'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 10), reused 21 (delta 7), pack-reused 0
Unpacking objects: 100% (27/27), 16.49 KiB | 1.50 MiB/s, done.
/content/federated-learning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 12.3 MB/s eta 0:00:00
  Installing build dependencies

In [3]:
import flwr as fl
from config import NUM_CLIENTS, DEVICE
from model import Net, VGG16
from main import simulate_cifar

## Centralize training

In [ ]:
from dataloader import load_cifars
trainloaders, valloaders, testloader = load_cifars(1)

trainloader = trainloaders[0]
valloader = valloaders[0]
net = VGG16().to(DEVICE)

for epoch in range(10):
    net.train_epoch(trainloader, 1)
    loss, accuracy = net.test(valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = net.test(testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: train loss 0.05028054490685463, accuracy 0.3868888888888889
Epoch 1: validation loss 0.04147230490446091, accuracy 0.5456
Epoch 1: train loss 0.033639177680015564, accuracy 0.6148
Epoch 2: validation loss 0.026561914348602295, accuracy 0.7034
Epoch 1: train loss 0.02576061710715294, accuracy 0.7148666666666667
Epoch 3: validation loss 0.02317628452181816, accuracy 0.744
Epoch 1: train loss 0.020628293976187706, accuracy 0.7752222222222223
Epoch 4: validation loss 0.021399199867248534, accuracy 0.771
Epoch 1: train loss 0.017199356108903885, accuracy 0.8127111111111112
Epoch 5: validation loss 0.01969984709620476, accuracy 0.7902
Epoch 1: train loss 0.0143361184746027, accuracy 0.8449333333333333
Epoch 6: validation loss 0.019189575929939747, accuracy 0.8004
Epoch 1: train loss 0.012040548026561737, accuracy 0.8689333333333333
Epoch 7: validation loss 0.017127903091907502, accuracy 0.8216
Epoch 1: train

## Fed avg with parameter initialization

In [4]:
net = Net().to(DEVICE)
params = net.get_parameters()

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

simulate_cifar(strategy, net)

100%|██████████| 170498071/170498071 [00:03<00:00, 45698569.78it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


INFO flwr 2023-06-09 11:48:46,061 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-06-09 11:48:49,753	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-09 11:48:52,149 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3930980352.0, 'memory': 7861960704.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3930980352.0, 'memory': 7861960704.0}
INFO flwr 2023-06-09 11:48:52,159 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-09 11:48:52,167 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-09 11:48:52,178 | server.py:88 | E

(launch_and_fit pid=2215) [Client 1] fit, config: {}
(launch_and_fit pid=2215) Epoch 1: train loss 0.06480904668569565, accuracy 0.23177777777777778


(pid=2294) 2023-06-09 11:49:08.203330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(pid=3477) 2023-06-09 11:51:32.868266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=3477) [Client 6] evaluate, config: {}


(pid=3539) 2023-06-09 11:51:39.782323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=3539) [Client 3] evaluate, config: {}


DEBUG flwr 2023-06-09 11:51:45,622 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-06-09 11:51:45,627 | server.py:147 | FL finished in 173.43467345199997
INFO:flwr:FL finished in 173.43467345199997
INFO flwr 2023-06-09 11:51:45,631 | app.py:218 | app_fit: losses_distributed [(1, 0.06227598198254903), (2, 0.054994750658671064), (3, 0.052339993794759114)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06227598198254903), (2, 0.054994750658671064), (3, 0.052339993794759114)]
INFO flwr 2023-06-09 11:51:45,633 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-09 11:51:45,635 | app.py:220 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-06-09 11:51:45,637 | app.py:221 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-06-09 11:51:45,638 | app.py:222 | app

In [8]:
from flwr.common import Metrics
from client import client_fn
from typing import List, Tuple
from dataloader import load_cifars

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)


trainloaders, valloaders, testloader = load_cifars(NUM_CLIENTS)

client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}


# Start simulation
fl.simulation.start_simulation(
    client_fn=lambda cid: client_fn(cid, net, trainloaders, valloaders),

    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)


Files already downloaded and verified
Files already downloaded and verified


INFO flwr 2023-06-09 12:17:18,805 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-06-09 12:17:24,111	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-09 12:17:25,853 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3922061721.0, 'memory': 7844123444.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3922061721.0, 'memory': 7844123444.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0}
INFO flwr 2023-06-09 12:17:25,859 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-09 12:17:25,862 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=10693) 2023-06-09 12:17:31.253294

(launch_and_get_parameters pid=10693) [Client 9] get_parameters


(pid=10807) 2023-06-09 12:17:46.973455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=10807) [Client 4] fit, config: {}
(launch_and_fit pid=10807) Epoch 1: train loss 0.06477593630552292, accuracy 0.2222222222222222


...

(launch_and_evaluate pid=16700) [Client 6] evaluate, config: {}


(pid=16771) 2023-06-09 12:29:59.472137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=16771) [Client 5] evaluate, config: {}


DEBUG flwr 2023-06-09 12:30:04,263 | server.py:182 | evaluate_round 5 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 5 results and 0 failures
INFO flwr 2023-06-09 12:30:04,270 | server.py:147 | FL finished in 749.4524640219997
INFO:flwr:FL finished in 749.4524640219997
INFO flwr 2023-06-09 12:30:04,275 | app.py:218 | app_fit: losses_distributed [(1, 0.06282049536705017), (2, 0.053854376268386836), (3, 0.05124085354804993), (4, 0.04943619599342346), (5, 0.04769706659317017)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06282049536705017), (2, 0.053854376268386836), (3, 0.05124085354804993), (4, 0.04943619599342346), (5, 0.04769706659317017)]
INFO flwr 2023-06-09 12:30:04,277 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-09 12:30:04,284 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.3048), (2, 0.37760000000000005), (3, 0.40360000000000007), (4, 0.41200000000000003), (5, 0.43)]}
I

History (loss, distributed):
	round 1: 0.06282049536705017
	round 2: 0.053854376268386836
	round 3: 0.05124085354804993
	round 4: 0.04943619599342346
	round 5: 0.04769706659317017
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.3048), (2, 0.37760000000000005), (3, 0.40360000000000007), (4, 0.41200000000000003), (5, 0.43)]}